In [ ]:
import tensorflow as tf
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.image as img
import PIL
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from PIL import Image
import random 
from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.layers import Input , MaxPool2D , Conv2D , Flatten , Dense , Dropout 
from tensorflow.keras import optimizers 
from tensorflow.keras.applications.inception_v3 import InceptionV3 
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.applications.vgg16 import VGG16 
from tensorflow.keras.applications.xception import Xception 
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.applications.vgg19 import VGG19 
from glob import glob 
import pathlib 
import os 
from sklearn.preprocessing import LabelEncoder  
from sklearn.utils import resample  
from keras.utils import to_categorical  
from sklearn.model_selection import train_test_split 
np.random.seed(101)

In [ ]:
# reading the data from the csv file 
skin_df = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')

#getting the image path for the images using the csv 
image_path = {os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join('/kaggle/input/skin-cancer-mnist-ham10000/', '*', '*.jpg'))}

# adding the images to the path obtained above 
skin_df['path'] = skin_df['image_id'].map(image_path.get)

In [ ]:
# using the above image path in order to add the images to the dataframe 
# via numpy reshape
skin_df['image'] = skin_df['path'].map(lambda x: np.asarray(Image.open(x).resize((128,128))))

In [ ]:
# checking the first 30 data entries from the csv 
skin_df.head(30)

In [ ]:
# using label encoder in order to grab the different classes of images from the 
# dataset and the csv
le = LabelEncoder()
le.fit(skin_df['dx'])
LabelEncoder()
print(list(le.classes_))

In [ ]:
skin_df['label'] = le.transform(skin_df['dx'])
print(skin_df.sample(10))

In [ ]:
# plotting the different relations for exloratory data analysis in the given 
# dataset for understanding the trends :-
fig = plt.figure(figsize=(15,10))

# Count by Cell Type
ax1 = fig.add_subplot(221)
skin_df['dx'].value_counts().plot(kind='bar', ax=ax1)
ax1.set_ylabel('Count')
ax1.set_title('Cell Type')

# Count by Sex
ax2 = fig.add_subplot(222)
skin_df['sex'].value_counts().plot(kind='bar', ax=ax2)
ax2.set_ylabel('Count', size=15)
ax2.set_title('Sex')

# Count by Localization (region)
ax3 = fig.add_subplot(223)
skin_df['localization'].value_counts().plot(kind='bar')
ax3.set_ylabel('Count', size=12)
ax3.set_title('Localization')

# Count by Age
ax4 = fig.add_subplot(224)
sample_age = skin_df[pd.notnull(skin_df['age'])]
sns.distplot(sample_age['age'], color='red')
ax4.set_title('Age')

plt.tight_layout()
plt.show()

In [ ]:
print(skin_df['label'].value_counts())

In [ ]:
df_0 = skin_df[skin_df['label'] == 0]
df_1 = skin_df[skin_df['label'] == 1]
df_2 = skin_df[skin_df['label'] == 2]
df_3 = skin_df[skin_df['label'] == 3]
df_4 = skin_df[skin_df['label'] == 4]
df_5 = skin_df[skin_df['label'] == 5]
df_6 = skin_df[skin_df['label'] == 6]

In [ ]:
# resampling 800 samples from each class with replacement in order to 
# balance the imbalanced dataset 

n_samples = 800
df_0_balanced = resample(df_0, replace=True, n_samples=n_samples, random_state=101)
df_1_balanced = resample(df_1, replace=True, n_samples=n_samples, random_state=101)
df_2_balanced = resample(df_2, replace=True, n_samples=n_samples, random_state=101)
df_3_balanced = resample(df_3, replace=True, n_samples=n_samples, random_state=101)
df_4_balanced = resample(df_4, replace=True, n_samples=n_samples, random_state=101)
df_5_balanced = resample(df_5, replace=True, n_samples=n_samples, random_state=101)
df_6_balanced = resample(df_6, replace=True, n_samples=n_samples, random_state=101)

In [ ]:
# combining all the above splitted dataframes into a single dataframe for 
# processing. 
skin_df_balanced = pd.concat([df_0_balanced, df_1_balanced, df_2_balanced, df_3_balanced, df_4_balanced, df_5_balanced, df_6_balanced])
skin_df_balanced

In [ ]:
# checking whether the classes are now balanced :- 
print(skin_df_balanced['label'].value_counts())

In [ ]:
n_samples = 5 # number of samples for plotting
# Plotting
fig, m_axs = plt.subplots(7, n_samples, figsize = (4*n_samples, 3*7))
for n_axs, (type_name, type_rows) in zip(m_axs, 
                                         skin_df_balanced.sort_values(['dx']).groupby('dx')):
    n_axs[0].set_title(type_name)
    
    for c_ax, (_, c_row) in zip(n_axs, type_rows.sample(n_samples, random_state=1234).iterrows()):
        c_ax.imshow(c_row['image'])
        c_ax.axis('off')

In [ ]:
# reshaping the images in order for better processing 
X = np.asarray(skin_df_balanced['image'].tolist())
X = X/255
Y = skin_df_balanced['label']
Y_cat = to_categorical(Y, num_classes=7)
Y_cat

In [ ]:
# getting the train test and validation splits , here the testing splitted data 
# will be used for validation 

x_train, x_test, y_train, y_test = train_test_split(X, Y_cat, test_size=0.25)

In [ ]:
#checking for the tensor shapes of the images :-

skin_df_balanced['image'].map(lambda x: x.shape).value_counts()

In [ ]:

'''
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)
'''



In [ ]:
#ALL the models Initiation and Compilation 

####
# INCEPTION_V3 MODEL :-
####

mod_InceptionV3 = InceptionV3(weights = "imagenet" , include_top = False , input_shape = (128 , 128 , 3))

for layer in mod_InceptionV3.layers:
    layer.trainable = False


x = mod_InceptionV3.output
x = Flatten()(x)
x = Dropout(0.35)(x)
x = Dense(7 , activation = 'softmax')(x)

model_InceptionV3 = Model(inputs = mod_InceptionV3.input , outputs = x)

model_InceptionV3.compile(loss = tf.keras.losses.CategoricalCrossentropy() , 
optimizer = optimizers.Adam(learning_rate = 0.001) , 
metrics = ['accuracy'])


####
# Xception MODEL :-
####
mod_Xception = Xception(weights = "imagenet" , include_top = False , input_shape = (128 , 128 , 3))
for layer in mod_Xception.layers:
    layer.trainable = False

m = mod_Xception.output
m = Flatten()(m)
m = Dropout(0.35)(m)
m = Dense(7 , activation = 'softmax')(m)

model_Xception = Model(inputs = mod_Xception.input , outputs = m)

model_Xception.compile(loss = tf.keras.losses.CategoricalCrossentropy() , 
optimizer = optimizers.Adam(learning_rate = 0.001) , 
metrics = ['accuracy'])


####
# DenseNet201 MODEL :-
####

mod_DenseNet201 = DenseNet201(weights = "imagenet" , include_top = False , input_shape = (128 , 128 , 3))

for layer in mod_DenseNet201.layers:
    layer.trainable = False


z = mod_DenseNet201.output
z = Flatten()(z)
z = Dropout(0.35)(z)
z = Dense(7 , activation = 'softmax')(z)
model_DenseNet201 = Model(inputs = mod_DenseNet201.input , outputs = z)

model_DenseNet201.compile(loss = tf.keras.losses.CategoricalCrossentropy() , 
optimizer = optimizers.Adam(learning_rate = 0.001) , 
metrics = ['accuracy'])




####
# VGG16 MODEL :-
####

mod_VGG16 = VGG16(weights = "imagenet" , include_top = False , input_shape = (128 , 128 , 3))

for layer in mod_VGG16.layers:
    layer.trainable = False


a = mod_VGG16.output
a = Flatten()(a)
a = Dropout(0.35)(a)
a = Dense(7 , activation = 'softmax')(a)

model_VGG16 = Model(inputs = mod_VGG16.input , outputs = a)

model_VGG16.compile(loss = tf.keras.losses.CategoricalCrossentropy() , 
                  optimizer = optimizers.Adam(learning_rate = 0.001) , 
                  metrics = ['accuracy'])




####
# NASNetMobile MODEL :-
####

mod_NASNET = NASNetMobile(weights = "imagenet" , include_top = False , input_shape = (128 , 128 , 3))

for layer in mod_NASNET.layers:
    layer.trainable = False


b = mod_NASNET.output
b = Flatten()(b)
b = Dropout(0.35)(b)
b = Dense(7 , activation = 'softmax')(b)

model_NASNet = Model(inputs = mod_NASNET.input , outputs = b)

model_NASNet.compile(loss = tf.keras.losses.CategoricalCrossentropy() , 
                  optimizer = optimizers.Adam(learning_rate = 0.001) , 
                  metrics = ['accuracy'])

In [ ]:
def matplot_plotting(tr_acc , tr_loss , val_acc , val_loss ):
    epochs = [i+1 for i in range(len(tr_acc))]

    plt.figure(figsize=(20, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, tr_loss, 'r', label='Train Loss')
    plt.plot(epochs, val_loss, 'g', label='Valid Loss')
    plt.title('Loss')
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')

    plt.subplot(1, 2, 2)
    plt.plot(epochs, tr_acc, 'r', label='Train Accuracy')
    plt.plot(epochs, val_acc, 'g', label='Valid Accuracy')
    plt.title('Accuracy')
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')

    plt.tight_layout()
    plt.show()

In [ ]:
# Model fitting of Inception_V3 and basic plotting :-

with tf.device('/device:GPU:0'):
    history_InceptionV3 = model_InceptionV3.fit(x_train , y_train , 
                        epochs = 50 , batch_size = 16 , 
                        validation_data = (x_test , y_test) , 
                        verbose = 1)
    

acc_InceptionV3 = history_InceptionV3.history['accuracy']
loss_InceptionV3 = history_InceptionV3.history['loss']
val_acc_InceptionV3 = history_InceptionV3.history['val_accuracy']
val_loss_InceptionV3 = history_InceptionV3.history['val_loss']

matplot_plotting(acc_InceptionV3 , loss_InceptionV3 , val_acc_InceptionV3 , val_loss_InceptionV3)

model_InceptionV3.save('Inception_V3_HAM10K.h5')

score_InceptionV3 = model_InceptionV3.evaluate(x_test , y_test)
print("Test Accuracy InceptionV3 : \t" , score_InceptionV3[1]) 

In [ ]:
# Model fitting of Xception and basic plotting :-

with tf.device('/device:GPU:0'):
    history_Xception = model_Xception.fit(x_train , y_train , 
                        epochs = 50 , batch_size = 16 , 
                        validation_data = (x_test , y_test) , 
                        verbose = 1)
    

acc_Xception = history_Xception.history['accuracy']
loss_Xception = history_Xception.history['loss']
val_acc_Xception = history_Xception.history['val_accuracy']
val_loss_Xception = history_Xception.history['val_loss']

matplot_plotting(acc_Xception , loss_Xception , val_acc_Xception , val_loss_Xception)

model_Xception.save('Xception_HAM10K.h5')

score_Xception = model_Xception.evaluate(x_test , y_test)
print("Test Accuracy Xception : \t" , score_Xception[1]) 

In [ ]:
# Model fitting of VGG16 and basic plotting :- 

with tf.device('/device:GPU:0'):
    history_VGG16 = model_VGG16.fit(x_train , y_train , 
                        epochs = 50 , batch_size = 16 , 
                        validation_data = (x_test , y_test) , 
                        verbose = 1)
    
acc_VGG16 = history_VGG16.history['accuracy']
loss_VGG16 = history_VGG16.history['loss']
val_acc_VGG16 = history_VGG16.history['val_accuracy']
val_loss_VGG16 = history_VGG16.history['val_loss']

matplot_plotting(acc_VGG16 , loss_VGG16 , val_acc_VGG16 , val_loss_VGG16)

model_VGG16.save('VGG16_HAM10K.h5')

score_VGG16 = model_VGG16.evaluate(x_test , y_test)
print("Test Accuracy InceptionV3 : \t" , score_VGG16[1]) 

In [ ]:
# Model fitting of DenseNet and basic plotting :- 

with tf.device('/device:GPU:0'):
    history_DenseNet201 = model_DenseNet201.fit(x_train , y_train , 
                        epochs = 50 , batch_size = 16 , 
                        validation_data = (x_test , y_test) , 
                        verbose = 1)
    
acc_DenseNet201= history_DenseNet201.history['accuracy']
loss_DenseNet201 = history_DenseNet201.history['loss']
val_acc_DenseNet201 = history_DenseNet201.history['val_accuracy']
val_loss_DenseNet201 = history_DenseNet201.history['val_loss']

matplot_plotting(acc_DenseNet201 , loss_DenseNet201 , val_acc_DenseNet201 , val_loss_DenseNet201)

model_DenseNet201.save('DenseNet201_HAM10K.h5')

score_DenseNet201 = model_DenseNet201.evaluate(x_test , y_test) 
print("Test Accuracy InceptionV3 : \t" , score_DenseNet201[1]) 

In [ ]:
# Model fitting of NASNet and basic plotting :- 

with tf.device('/device:GPU:0'):
    history_NASNet = model_NASNet.fit(x_train , y_train , 
                        epochs = 50 , batch_size = 16 , 
                        validation_data = (x_test , y_test) , 
                        verbose = 1)
    
acc_NASNet = history_NASNet.history['accuracy']
loss_NASNet = history_NASNet.history['loss']
val_acc_NASNet = history_NASNet.history['val_accuracy']
val_loss_NASNet = history_NASNet.history['val_loss']

matplot_plotting(acc_NASNet , loss_NASNet , val_acc_NASNet , val_loss_NASNet)

model_NASNet.save('NASNet_HAM10K.h5')

score_NASNet = model_NASNet.evaluate(x_test , y_test)
print("Test Accuracy InceptionV3 : \t" , score_NASNet[1]) 

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": False}]])

# Add traces
fig.add_trace(
    go.Scatter( y= acc_InceptionV3, name="InceptionV3")
   
)

fig.add_trace(
    go.Scatter( y= acc_Xception, name="Xception")

)

fig.add_trace(
    go.Scatter( y= acc_VGG16, name="VGG16")
    
)

fig.add_trace(
    go.Scatter( y= acc_DenseNet201, name="DenseNet201")
    
)

fig.add_trace(
    go.Scatter( y= acc_NASNet, name="NASNetMobile")
    
)
# Add figure title
fig.update_layout(
    title_text="Accuracy of all Models :"
)

# Set x-axis title
fig.update_xaxes(title_text="Epoch")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Accuracy</b>", secondary_y=False)


fig.show()

In [ ]:
epochs = [i+1 for i in range(len(acc_InceptionV3))]

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc_InceptionV3, 'r', label='Inception_V3')
plt.plot(epochs, acc_Xception , 'g', label='Xception')
plt.plot(epochs, acc_VGG16, 'b', label='VGG16')
plt.plot(epochs, acc_DenseNet201, 'y', label='DenseNet201')
plt.plot(epochs, acc_NASNet, 'c', label='NASNet')
plt.title('Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy %')

plt.tight_layout()
plt.show()

In [ ]:
/kaggle/working/